In [116]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
from subprocess import check_output


In [117]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


data_train = pd.read_csv("../content/fashion-mnist_train1.csv")
data_test = pd.read_csv("../content/fmnist_test1.csv")


X = np.array(data_train.iloc[:, 1:])
y = to_categorical(np.array(data_train.iloc[:, 0]))


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)


X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))

In [118]:

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)


X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_train /= 255


X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_test = X_test.astype('float32')
X_test /= 255


X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
X_val = X_val.astype('float32')
X_val /= 255

In [119]:

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.2))

cnn1.add(Flatten())

cnn1.add(Dense(128, activation='relu'))
cnn1.add(Dense(10, activation='softmax'))

In [120]:
cnn1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [121]:
cnn1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 13, 13, 32)        0         
                                                                 
 flatten_3 (Flatten)         (None, 5408)              0         
                                                                 
 dense_6 (Dense)             (None, 128)               692352    
                                                                 
 dense_7 (Dense)             (None, 10)                1290      
                                                      

In [122]:
history1 = cnn1.fit(X_train, y_train,
          batch_size=256,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Epoch 1/10
188/188 [==============================] - 29s 152ms/step - loss: 0.5275 - accuracy: 0.8156 - val_loss: 0.3943 - val_accuracy: 0.8639
Epoch 2/10
188/188 [==============================] - 28s 150ms/step - loss: 0.3619 - accuracy: 0.8725 - val_loss: 0.3629 - val_accuracy: 0.8687
Epoch 3/10
188/188 [==============================] - 27s 145ms/step - loss: 0.3172 - accuracy: 0.8886 - val_loss: 0.3212 - val_accuracy: 0.8860
Epoch 4/10
188/188 [==============================] - 27s 144ms/step - loss: 0.2877 - accuracy: 0.8988 - val_loss: 0.3023 - val_accuracy: 0.8935
Epoch 5/10
188/188 [==============================] - 28s 150ms/step - loss: 0.2727 - accuracy: 0.9028 - val_loss: 0.2882 - val_accuracy: 0.8977
Epoch 6/10
188/188 [==============================] - 27s 144ms/step - loss: 0.2518 - accuracy: 0.9103 - val_loss: 0.2731 - val_accuracy: 0.9005
Epoch 7/10
188/188 [==============================] - 27s 143ms/step - loss: 0.2435 - accuracy: 0.9131 - val_loss: 0.2713 - val_ac

In [123]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=256)
val_batches = gen.flow(X_val, y_val, batch_size=256)

In [124]:
predict_x=cnn1.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

313/313 [==============================] - 3s 10ms/step


In [125]:
df_submission = pd.DataFrame(classes_x)
df_submission.to_csv('submission_final.csv', sep=",")


In [126]:
df = pd.read_csv('submission_final.csv')


In [127]:
df.index.rename('Id', inplace= True )

In [128]:
df.columns

Index(['Unnamed: 0', '0'], dtype='object')

In [129]:
df.columns = ['Id', 'Category']
df.drop (columns=df.columns [0], axis= 1 , inplace= True )

In [130]:
df.head()

,Category
Id,
0,0
1,1
2,2
3,6
4,3


In [131]:
df.to_csv('submission_final3.csv')

In [132]:
from google.colab import files
files.download(f"./submission_final3.csv") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>